In [1]:
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk

In [2]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

In [4]:
print(json_list[0])

{'reviewText': 'Spiritually and mentally inspiring! A book that allows you to question your morals and will help you discover who you really are!', 'unixReviewTime': 1355616000, 'reviewerID': 'A10000012B7CGYKOMPQ4L', 'reviewTime': '12 16, 2012', 'overall': 5.0, 'reviewerName': 'Adam', 'summary': 'Wonderful!', 'asin': '000100039X', 'helpful': [0, 0]}


In [5]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)
type(review_data)

list

In [6]:
import random
random_data = random.sample(review_data,len(review_data))
random_data[0]

{'overall': 5.0,
 'reviewText': 'The sheer literary pleasure of this masterpiece is something I\'ll miss for a long time. I don\'t think I can add much to one of the most reviewed book of all times but I will always keep in mind what Ursula K. LeGuin wrote about it in her "Rhythmic Patterning in The Lord of the Rings" article, that "It\'s a wonderful book to read aloud or (consensus by the children) listen to." and that "... it wants the living voice to speak it, to find its full beauty and power, its subtle music, its rhythmic vitality."After the end of this literary journey, I have a strong feeling, an inner sound that tells me I have yet to face many events that will be tied to LOTR in some way. This may well be the echoes of my past reading, and sometimes reading aloud; but even if that feeling is illusory, and the sound is void, I have to thank to Tolkien for having created such a repercussive narration. If "art is a lie that makes us realize truth" as Picasso said, then LOTR open

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samsung\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
reviewText = []
for i in range(len(random_data)):
    reviewText.append(random_data[i]["reviewText"])
print(reviewText[2:4])
len(reviewText)

["The book started out interesting, but then became such a recital of reports that the story of Marilyn's life turned dry and tedious.", "If you are reading this, you don't need a description, it is a classic and to be enjoyed for years. Not to mention FREE on Kindle...amazing."]


93184

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, max_df = 0.3, min_df = 2, lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(reviewText)
# max_df = 0.4 , min_df 0.01 , Accuracy = 49%
# max_df = 0.4 , min_df 0.0.005 , Accuracy = 53%
# max_df = 0.4 , min_df 0.0.004 , Accuracy = ?

# 40%제거 vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, max_df = 0.3, min_df = 2, lowercase=True, stop_words = 'english')
# default : (93184, 116502)
# (93184, 60901)

In [64]:
D.shape
# 행,렬 

(93184, 60901)

In [31]:
D = D.toarray()
print(D)
#리스트로 만듬

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [32]:
y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
for i in y_data:
    int_y_data.append((int(i),))
print(int_y_data[:10])

[(4,), (5,), (2,), (2,), (4,), (5,), (3,), (4,), (5,), (5,)]


In [33]:
y_data = np.array(MultiLabelBinarizer().fit_transform(int_y_data))

In [34]:
y_data.shape

(93184, 5)

In [35]:
print(y_data[:10])

[[0 0 0 1 0]
 [0 0 0 0 1]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 1]]


In [ ]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [93184, 60901])

w1 = tf.Variable(tf.truncated_normal([60901, 300], stddev = 0.1))
b1 = tf.Variable(tf.truncated_normal([300]))
w2 = tf.Variable(tf.truncated_normal([300, 5], stddev = 0.1))
b2 = tf.Variable(tf.truncated_normal([5]))

output =  tf.sigmoid(tf.matmul(x, w1) + b1)
hypothesis = tf.sigmoid(tf.matmul(output,w2)+b2)

cost = -tf.reduce_sum(y_data * tf.log(tf.clip_by_value(hypothesis,1e-10,1.0)))
train_step = tf.train.AdamOptimizer(0.01).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(200):
    sess.run(train_step, feed_dict={x:D})
    if i % 100 == 0:
        print(i, sess.run(cost, feed_dict={x:D}))
        
# 뉴럴넷
# 0.1 , 0.01 49%
# 0.1 , 0.005 53%
# 300 : 2034, 53%
# 300 : 60901 ??

In [ ]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y_data,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: D}))
# 정확도 측정

In [ ]:
print(sess.run(hypothesis, feed_dict={x: D}))
# 결과값

In [ ]:
print(sess.run(tf.argmax(hypothesis, 1), feed_dict={x: D}))
# 결과값 중에 높은 인덱스 가져오기